In [1]:
import pandas as pd
import numpy as np

In [2]:
excel = pd.read_excel('dados/Dicionário_Microdados_Enem_2019.xlsx',
                      skiprows=[0,1,2],
                      names=['variavel', 'descricao', 'categoria', 'desc-categoria', 'tamanho', 'tipo'])

In [3]:
excel['variavel'].where(excel['variavel'].str.startswith('DADO')).dropna()
# 0                                  DADOS DO PARTICIPANTE
# 65                                       DADOS DA ESCOLA
# 84        DADOS DOS PEDIDOS DE ATENDIMENTO ESPECIALIZADO
# 111          DADOS DOS PEDIDOS DE ATENDIMENTO ESPECÍFICO
# 120    DADOS DOS PEDIDOS DE RECURSOS ESPECIALIZADOS E...
# 189                 DADOS DO LOCAL DE APLICAÇÃO DA PROVA
# 198                              DADOS DA PROVA OBJETIVA
# 267                                     DADOS DA REDAÇÃO
# 282                 DADOS DO QUESTIONÁRIO SOCIOECONÔMICO

0                                  DADOS DO PARTICIPANTE
65                                       DADOS DA ESCOLA
84        DADOS DOS PEDIDOS DE ATENDIMENTO ESPECIALIZADO
111          DADOS DOS PEDIDOS DE ATENDIMENTO ESPECÍFICO
120    DADOS DOS PEDIDOS DE RECURSOS ESPECIALIZADOS E...
189                 DADOS DO LOCAL DE APLICAÇÃO DA PROVA
198                              DADOS DA PROVA OBJETIVA
267                                     DADOS DA REDAÇÃO
282                 DADOS DO QUESTIONÁRIO SOCIOECONÔMICO
Name: variavel, dtype: object

In [4]:
participante = excel.iloc[1:65]
escola = excel.iloc[66:84]
especializado = excel.iloc[85:111]
especifico = excel.iloc[112:120]
recurso = excel.iloc[121:189]
local_prova = excel.iloc[190:198]
prova = excel.iloc[199:267]
redacao = excel.iloc[268:282]
socioeconomico = excel.iloc[283:-10]

In [5]:
footer = excel.iloc[-10:]

In [6]:
def variaveis_e_categorias(df):
    variaveis = df.where(df['variavel'].notnull()).dropna(how='all')
    variaveis['categoria'] = variaveis['categoria'].notnull()
    variaveis['tamanho'] = variaveis['tamanho'].astype(np.int64)

    categorias = df.where(df['categoria'].notnull()).dropna(how='all').fillna(method='ffill')
    categorias['tamanho'] = categorias['tamanho'].astype(np.int64)
    return (
        #variaveis.drop(columns=['desc-categoria']).set_index('variavel').to_dict(orient='index')
        variaveis.drop(columns=['desc-categoria']),
        categorias
    )

In [7]:
loop = [
    ('participante', participante),
    ('escola', escola),
    ('especializado', especializado),
    ('especifico', especifico),
    ('recurso', recurso),
    ('local_prova', local_prova),
    ('prova', prova),
    ('redacao', redacao),
    ('socioeconomico', socioeconomico),
]

dados = {}
categorias = {}

def df_categoria_to_dict(df, *args, **kwargs):
    cat = df[['categoria', 'desc-categoria']]
    l = list(cat.set_index('categoria').to_records())
    return { str(item[0]): str(item[1]) for item in l }


for label, df in loop:
    v, c = variaveis_e_categorias(df)
    dados[label] = v.set_index('variavel')['descricao'].to_dict()
    
    categorias[label] = (
        c[['variavel','categoria', 'desc-categoria']].groupby('variavel').apply(df_categoria_to_dict)
    ).to_dict()

In [12]:
dados['especializado']

{'IN_BAIXA_VISAO': 'Indicador de baixa visão5',
 'IN_CEGUEIRA': 'Indicador de cegueira5',
 'IN_SURDEZ': 'Indicador de surdez5',
 'IN_DEFICIENCIA_AUDITIVA': 'Indicador de deficiência auditiva5',
 'IN_SURDO_CEGUEIRA': 'Indicador de surdo-cegueira5',
 'IN_DEFICIENCIA_FISICA': 'Indicador de deficiência física5',
 'IN_DEFICIENCIA_MENTAL': 'Indicador de deficiência mental5',
 'IN_DEFICIT_ATENCAO': 'Indicador de déficit de atenção5',
 'IN_DISLEXIA': 'Indicador de dislexia5',
 'IN_DISCALCULIA': 'Indicador de discalculia5',
 'IN_AUTISMO': 'Indicador de autismo5',
 'IN_VISAO_MONOCULAR': 'Indicador de visão monocular5',
 'IN_OUTRA_DEF': 'Indicador de outra deficiência ou condição especial5'}

In [9]:
categorias

{'participante': {'IN_TREINEIRO': {'1': 'Sim', '0': 'Não'},
  'TP_ANO_CONCLUIU': {'0': 'Não informado',
   '1': '2018',
   '2': '2017',
   '3': '2016',
   '4': '2015',
   '5': '2014',
   '6': '2013',
   '7': '2012',
   '8': '2011',
   '9': '2010',
   '10': '2009',
   '11': '2008',
   '12': '2007',
   '13': 'Antes de 2007'},
  'TP_COR_RACA': {'0': 'Não declarado',
   '1': 'Branca',
   '2': 'Preta',
   '3': 'Parda',
   '4': 'Amarela',
   '5': 'Indígena '},
  'TP_ENSINO': {'1': 'Ensino Regular',
   '2': 'Educação Especial - Modalidade Substitutiva',
   '3': 'Educação de Jovens e Adultos'},
  'TP_ESCOLA': {'1': 'Não Respondeu',
   '2': 'Pública',
   '3': 'Privada',
   '4': 'Exterior'},
  'TP_ESTADO_CIVIL': {'0': 'Não informado',
   '1': 'Solteiro(a)',
   '2': 'Casado(a)/Mora com companheiro(a)',
   '3': 'Divorciado(a)/Desquitado(a)/Separado(a)',
   '4': 'Viúvo(a)'},
  'TP_NACIONALIDADE': {'0': 'Não informado',
   '1': 'Brasileiro(a)',
   '2': 'Brasileiro(a) Naturalizado(a)',
   '3': 'Estra

In [10]:
import json
with open('dicionario.json', 'w') as f:
    json.dump(dados, f, ensure_ascii=False, indent=2)
    
with open('categorias.json', 'w') as f:
    json.dump(categorias, f, ensure_ascii=False, indent=2)